In [17]:
import numpy as np
import cv2
import pandas as pd
import os
import pickle

In [18]:
face_detection_model = './models/res10_300x300_ssd_iter_140000_fp16.caffemodel'
face_detection_prot = './models/deploy.prototxt.txt'
face_descriptor = './models/openface.nn4.small2.v1.t7'

# load models
detector_model = cv2.dnn.readNetFromCaffe(face_detection_prot, face_detection_model)
descriptor_model = cv2.dnn.readNetFromTorch(face_descriptor)

In [19]:
!dir people-images

 Volume in drive D is New Volume
 Volume Serial Number is A8DC-3007

 Directory of D:\Archive\Record\django_open_cv\projects\01-image-processing\people-images

01/12/2022  10:40 PM    <DIR>          .
01/12/2022  10:48 PM    <DIR>          ..
01/12/2022  10:34 PM    <DIR>          Elon Musk
01/12/2022  10:36 PM    <DIR>          messi
01/12/2022  10:39 PM    <DIR>          ronaldo
01/12/2022  10:42 PM    <DIR>          shahrokh khan
               0 File(s)              0 bytes
               6 Dir(s)  67,846,619,136 bytes free


In [20]:
# sample image
img = cv2.imread('./images/1.jpeg')
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

if len(detections) > 0:
    i = np.argmax(detections[0,0,:,2])
    confidence = detections[0,0,i,2]
    if confidence > 0.5:
        box = detections[0,0,i,3:7] * np.array([w,h,w,h])
        (startx, starty, endx, endy) = box.astype('int')
        # step 2: feature extraction
        roi = image[starty:endy, startx:endx].copy()
        # get the face descriptor
        faceblob = cv2.dnn.blobFromImage(roi, 1/255, (96,96), (0,0,0), swapRB=True, crop=True)
        descriptor_model.setInput(faceblob)
        vectors = descriptor_model.forward()

In [44]:
def helper(image_path):
    img = cv2.imread(image_path)
    # step-1: face detection
    image = img.copy()
    h,w = image.shape[:2]
    img_blob = cv2.dnn.blobFromImage(image,1,(300,300),(104,177,123),swapRB=False,crop=False)
    # set the input
    detector_model.setInput(img_blob)
    detections = detector_model.forward()

    if len(detections) > 0:
        i = np.argmax(detections[0,0,:,2])
        confidence = detections[0,0,i,2]
        if confidence > 0.5:
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startx,starty,endx,endy) = box.astype('int')
            # step-2: Feature Extraction or Embedding
            roi = image[starty:endy,startx:endx].copy()
            # get the face descriptors
            faceblob = cv2.dnn.blobFromImage(roi,1/255,(96,96),(0,0,0),swapRB=True,crop=True)
            descriptor_model.setInput(faceblob)
            vectors = descriptor_model.forward()
            
            return vectors
    return None

In [47]:
data = dict(data=[], label=[])

folders = os.listdir('people-images')
for folder in folders:
    filenames = os.listdir('people-images/{}'.format(folder))
    for filename in filenames:
        try:
            vector = helper('./people-images/{}/{}'.format(folder, filename))
            if vector is not None:
                data['data'].append(vector)
                data['label'].append(folder)
        except:
            print('error')
            
            

In [48]:
data.keys()

dict_keys(['data', 'label'])

In [49]:
pd.Series(data['label']).value_counts()

Elon Musk        10
messi            10
ronaldo          10
shahrokh khan    10
dtype: int64

In [50]:
pickle.dump(data,open('data_face_features.pickle', mode='wb'))